In [1]:
# Librerías
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from astroquery.mpc import MPC

In [2]:
# Conexión del API con el programa
url = 'https://cobs.si/api/obs_list.api?des=C/2023 A3&format=json&from_date=&to_date=&exclude_faint=False&exclude_not_accurate=False'
response = requests.get(url) 

if response.status_code == 200:
    content = response.json()
    print('✅ Base de datos actualizada.')
else:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response.status_code}\n{response.content}')

In [ ]:
# Creación de Dataset
cometa_df = pd.DataFrame(content['objects'])
cometa_df.sort_values('obs_date', ascending=False).info()

In [ ]:
# Sin filtrar la información
filas,columnas = cometa_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 1148
Variables: 47


In [ ]:
# Base de datos arrojada por la API
cometa_df.sample(5)

,type,obs_date,comet,observer,location,extinction,obs_method,comet_visibility,magnitude,conditions,...,magnitude_error,comparison_star_magnitude,pixel_size_x,pixel_size_y,pixel_size_unit,obs_comment,obs_sky_quality,obs_sky_quality_method,reference_star_names,date_added
968,C,2024-01-17 17:39:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Michael', 'last_name': 'Mattia...","Swan Hill, Victoria, Australia",None,"{'key': 'Z', 'name': 'CCD Visual equivalent', ...",None,14.4,None,...,None,None,None,None,None,,None,NaN,,2024-01-18 21:27:25
425,C,2024-05-13 21:04:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Mieczyslaw Leszek', 'last_name...",None,None,"{'key': 'V', 'name': 'Johnson/Bessel/Kron/Cous...",None,10.1,None,...,0.01,None,2.9,2.9,s,Temparature: +9 C; Humidity 23%,None,NaN,,2024-05-23 15:37:30
193,V,2024-06-10 21:36:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Alexandre', 'last_name': 'Amor...",None,None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,9.9,None,...,None,None,None,None,None,,None,NaN,,2024-06-11 11:54:22
837,C,2024-04-03 14:09:35,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Andrew', 'last_name': 'Pearce'...",None,None,"{'key': 'C', 'name': 'Unfiltered total CCD mag...",None,11.5,None,...,0.07,None,1.6,1.6,s,,None,NaN,,2024-04-04 09:11:42
427,V,2024-05-13 20:40:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Eryk', 'last_name': 'Banach', ...",None,None,"{'key': 'M', 'name': 'Modified-Out method', 's...",None,10.1,None,...,None,None,None,None,None,,20.5,3.0,,2024-05-13 22:17:05


In [ ]:
# Métodos de observación
cometa_df.obs_method.apply(lambda registro: f'{registro['key']}: {registro['name']}').value_counts()

obs_method
C: Unfiltered total CCD magnitude            338
Z: CCD Visual equivalent                     292
S: In-Out method                             193
M: Modified-Out method                       174
V: Johnson/Bessel/Kron/Cousins V with CCD     76
B: Simple Out-Out method                      19
k: Kron/Cousins R with CCD                    18
H: Kron/Cousins I with CCD                    17
D: Johnson/Bessel/Kron/Cousins B with CCD     10
c: Unfiltered nuclear CCD magnitude            8
d: Astrodon G filter                           2
E: Extrafocal-Extinction method                1
Name: count, dtype: int64

In [ ]:
# Con valores nulos
filas,columnas = cometa_df.shape
print(f'Registros: {filas}\nVariables: {columnas + 1}')

Registros: 1148
Variables: 48


In [ ]:
# Tratamiento de los datos de interés
cometa_df['obs_method_key'] = cometa_df.obs_method.apply(lambda registro: registro['key'])
cometa_df['obs_date'] = pd.to_datetime(cometa_df.obs_date)
cometa_df['magnitude'] = pd.to_numeric(cometa_df.magnitude)

In [ ]:
# Info de nuevo data frame
curva_de_luz_df = cometa_df[['obs_method_key', 'obs_date', 'magnitude']]
curva_de_luz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1148 entries, 0 to 1147
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  1148 non-null   object        
 1   obs_date        1148 non-null   datetime64[ns]
 2   magnitude       1148 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 27.0+ KB


In [ ]:
# Data Frame de la curva de luz
curva_de_luz_df.sample(5)

,obs_method_key,obs_date,magnitude
806,M,2024-04-09 22:04:48,10.8
165,M,2024-06-19 21:35:59,10.3
1066,Z,2023-05-26 23:31:12,17.0
232,M,2024-06-05 23:45:36,10.4
309,V,2024-05-29 13:12:00,10.7


In [ ]:
# Gráfica del los datos
labels = {'obs_date':'Observation Date','magnitude':'Apparent total magnitude', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_df, x='obs_date', y='magnitude', color='obs_method_key', template= 'plotly_dark', labels= labels, title='Lightcurve of comet C/2023 A3 (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [ ]:
# Variables a graficar
tiempo = curva_de_luz_df.obs_date
obs_method = curva_de_luz_df.obs_method_key.unique()

# Inicializar la figura con subplots
fig = make_subplots(rows=4, cols=3, subplot_titles= obs_method)

# Agregar trazas a los subplots
for metodo, fila, columna  in zip(obs_method, [1]*3 + [2]*3 + [3]*3, [1,2,3]*3):
    fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_df.magnitude[curva_de_luz_df.obs_method_key == metodo], mode="markers",name = metodo), row = columna, col = fila)

# Actualizar el diseño de los subplots
fig.update_layout(title_text = 'Lightcurve of comet C/2023 A3 (Tsuchinshan-ATLAS)', template = 'plotly_dark')
fig.update_yaxes(autorange="reversed")

# Mostrar la figura
fig.show()

In [41]:
# Ephemeris
eph = MPC.get_ephemeris('C/2023 A3')
ephemeris_df = eph.to_pandas()
ephemeris_df.columns = ephemeris_df.columns.str.lower()
ephemeris_df

,date,ra,dec,delta,r,elongation,phase,v,proper motion,direction
0,2024-08-24 15:57:59,161.624167,-2.255556,1.858,0.923,15.4,16.9,9.1,25.58,216.9
1,2024-08-25 15:57:59,161.520417,-2.392500,1.844,0.904,14.7,16.5,9.0,25.91,217.2
2,2024-08-26 15:57:59,161.415000,-2.530556,1.830,0.885,14.0,16.1,8.9,26.25,217.5
3,2024-08-27 15:57:59,161.307500,-2.670000,1.815,0.865,13.4,15.8,8.8,26.59,217.8
4,2024-08-28 15:57:59,161.197500,-2.810833,1.800,0.846,12.9,15.5,8.7,26.94,218.1
5,2024-08-29 15:57:59,161.085000,-2.952500,1.783,0.826,12.4,15.2,8.6,27.30,218.5
6,2024-08-30 15:57:59,160.970417,-3.095556,1.767,0.807,12.0,15.1,8.5,27.65,219.0
7,2024-08-31 15:57:59,160.852917,-3.239167,1.749,0.788,11.7,15.1,8.4,28.00,219.4
8,2024-09-01 15:57:59,160.732917,-3.383889,1.731,0.768,11.4,15.1,8.3,28.34,219.8
9,2024-09-02 15:57:59,160.610417,-3.529444,1.711,0.749,11.3,15.3,8.2,28.66,220.3


In [45]:
ephemeris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           21 non-null     object 
 1   ra             21 non-null     float64
 2   dec            21 non-null     float64
 3   delta          21 non-null     float64
 4   r              21 non-null     float64
 5   elongation     21 non-null     float64
 6   phase          21 non-null     float64
 7   v              21 non-null     float64
 8   proper motion  21 non-null     float64
 9   direction      21 non-null     float64
dtypes: float64(9), object(1)
memory usage: 1.8+ KB


In [48]:
ephemeris_df.date = pd.to_datetime(ephemeris_df.date)
ephemeris_df.date = ephemeris_df.date.dt.date
ephemeris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           21 non-null     object 
 1   ra             21 non-null     float64
 2   dec            21 non-null     float64
 3   delta          21 non-null     float64
 4   r              21 non-null     float64
 5   elongation     21 non-null     float64
 6   phase          21 non-null     float64
 7   v              21 non-null     float64
 8   proper motion  21 non-null     float64
 9   direction      21 non-null     float64
dtypes: float64(9), object(1)
memory usage: 1.8+ KB


In [51]:
curva_de_luz_ephemeris_df = ephemeris_df[['date', 'delta','r', 'phase']]
curva_de_luz_ephemeris_df

,date,delta,r,phase
0,2024-08-24,1.858,0.923,16.9
1,2024-08-25,1.844,0.904,16.5
2,2024-08-26,1.830,0.885,16.1
3,2024-08-27,1.815,0.865,15.8
4,2024-08-28,1.800,0.846,15.5
5,2024-08-29,1.783,0.826,15.2
6,2024-08-30,1.767,0.807,15.1
7,2024-08-31,1.749,0.788,15.1
8,2024-09-01,1.731,0.768,15.1
9,2024-09-02,1.711,0.749,15.3
